<a href="https://colab.research.google.com/github/adarshnitt/Deep-Learning/blob/main/Potatao_Disease_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Data Loading </h1>

In [5]:
# we will use data from Kaggle, so fist match colab and Kaggle
! pip install -q kaggle  # q : quit no verbose 

In [6]:
# downloading token from kaggle and saving in local computer then importing at new dir (~/.kaggle)
# path for token: Kaggle=>Account=>API_Token
from google.colab import files
files.upload()
# data will go to content section firstly

Saving kaggle.json to kaggle (2).json


{'kaggle.json': b'{"username":"adarshnitt","key":"6d32d5af10db0f45742b74aaa798b5e9"}'}

In [7]:
# imported data from kaggle to g-cloud in directory ~/.kaggle
! mkdir ~/.kaggle  # ~ home dir

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [8]:
# moving file kaggle.json to our new dir(~/.kagle)
# Ensure kaggle.json is in the location ~/.kaggle/kaggle.json to use the API.
! cp kaggle.json ~/.kaggle/

In [9]:
# lets see how many datasets are freely available in kaggle
! kaggle datasets list # getting info about all datasets.

ref                                                         title                                              size  lastUpdated          downloadCount  
----------------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  
gpreda/reddit-vaccine-myths                                 Reddit Vaccine Myths                              237KB  2021-12-12 11:59:54          18347  
crowww/a-large-scale-fish-dataset                           A Large Scale Fish Dataset                          3GB  2021-04-28 17:03:01          11029  
imsparsh/musicnet-dataset                                   MusicNet Dataset                                   22GB  2021-02-18 14:12:19           5606  
dhruvildave/wikibooks-dataset                               Wikibooks Dataset                                   2GB  2021-10-22 10:48:21           3913  
nickuzmenkov/nih-chest-xrays-tfrecords                      NIH Chest X-rays

In [10]:
# securing our kaggle api 
! chmod 600 ~/.kaggle/kaggle.json
#  chmos to 600 means only owner can read and write access of directory ~/.kaggle/kaggale.json.

In [11]:
# m-1 to attach data in our newly built directory 
! kaggle competion download -c kaggle datasets download -d muhammadardiputra/potato-leaf-disease-dataset
# Your Kaggle API key is readable by other users on this system! To fix this, you can run 'chmod 600 /root/.kaggle/kaggle.json'
#  chmos to 600 means only owner can read and write access of directory ~/.kaggle/kaggale.json.

usage: kaggle [-h] [-v] {competitions,c,datasets,d,kernels,k,config} ...
kaggle: error: argument command: invalid choice: 'competion' (choose from 'competitions', 'c', 'datasets', 'd', 'kernels', 'k', 'config')


In [12]:
# downloading data at content/our_data
! kaggle datasets download -d muhammadardiputra/potato-leaf-disease-dataset
# -d is same as -v(verbosity)

potato-leaf-disease-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [13]:
# importing libraries
import pandas as pd
import numpy as np

In [15]:
# unzipping zip data file
! unzip "/content/potato-leaf-disease-dataset.zip"

Archive:  /content/potato-leaf-disease-dataset.zip
replace Potato/Test/Potato___Early_blight/034959c1-f1e8-4a79-a6d5-3c1d14efa2f3___RS_Early.B 7136.JPG? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [25]:
# listing dir in test data
os.listdir("/content/Potato/Test") 

['Potato___Late_blight', 'Potato___Early_blight', 'Potato___healthy']

In [36]:
# no of images in each dir of test data
def info(d1):
  for samp in os.listdir(d1):
    subsamp=os.path.join(d1,samp)
    l=len(os.listdir(subsamp))
    print(l, " and samp type is -",samp,"***",d1[-5:])
  pass

In [81]:
info("/content/Potato/Valid")

100  and samp type is - Potato___Late_blight *** Valid
100  and samp type is - Potato___Early_blight *** Valid
100  and samp type is - Potato___healthy *** Valid


In [38]:
info("/content/Potato/Train")

300  and samp type is - Potato___Late_blight *** Train
300  and samp type is - Potato___Early_blight *** Train
300  and samp type is - Potato___healthy *** Train


In [ ]:
# practice notes
"""
import os
import shutil
shutil.rmtree("/content/Potato/Train/new_train")
os.mkdir()
os.rmdir()
# to move a dqata from one folder to anothwr
os.listdir("")
os.path.join(a,b) : give a returned path
random.choice(): choose randomly any one from given data.
""

<h1> Preprocessing </h1>


In [77]:
# real time image augmentation: scaling, shearing,rotaion etc
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def augment_train(path):
  "for train image"
  model=ImageDataGenerator(rotation_range=360,
                           shear_range=10, # angular ration float
                           zoom_range=0.3, # % zoom augmentation
                           rescale=1/255,  # rescaling (0-1)
                           horizontal_flip=True,
                           vertical_flip=True)
  # generating batches of augmenting data
  image=model.flow_from_directory(path,
                                  target_size=(224,224,3), # output augmented image shape 
                                  class_mode="binary",
                                  batch_size=50)
  return image

In [72]:
def augment_val(path):
  " for validation data"
  model=ImageDataGenerator(rescale=1/255)
  img=model.flow_from_directory(directory=path,
                                target_size=(224,224,3),
                                batch_size=50,
                                class_mode="binary")
  return img

In [80]:
data_train_augment=augment_train("/content/Potato/Train")
data_val_augment=augment_val("/content/Potato/Valid")

Found 900 images belonging to 3 classes.
Found 300 images belonging to 3 classes.


In [17]:
import pandas as pd
import numpy as np
#! pip install  tensorflow-gpu
import tensorflow as tf
from tensorflow import keras 
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import seaborn as sns

In [18]:
from tensorflow.keras import layers,models
from keras import Sequential

<h1> Model Building</h1>

In [48]:
from keras.layers import Conv2D,MaxPool2D,Dropout,Flatten,Dense,BatchNormalization, GlobalAvgPool2D
from keras.models import Sequential
from tensorflow.keras.preprocessing.image import load_img, img_to_array

<h1> CNN</h1>

In [62]:
model=Sequential()
# input shape is x then output will be ((x-f+2p)/s)+1.

model.add(Conv2D(filters=16,  #  16 specific feature based filters
                 kernel_size=(2,2), # filter size
                 activation="relu",
                 padding="same",  # final size will same as input
                 input_shape=(224,224,3)))  # input image shape
model.add(Conv2D(filters=32,kernel_size=(3,3),
                 activation="relu"))
model.add(MaxPool2D(pool_size=(2,2))) # overfitting avoid use maxpol
model.add(Conv2D(filters=64,
                 kernel_size=(2,2),
                 activation="relu"))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(rate=0.2))  # 20% nodes are cold
model.add(Flatten())
model.add(Dense(units=64,
                activation="relu"))  # dense neural nn with 64 activation nodes
model.add(Dropout(rate=0.3))
model.add(Dense(units=1,
                activation="sigmoid"))

In [63]:
model.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_22 (Conv2D)          (None, 224, 224, 16)      208       
                                                                 
 conv2d_23 (Conv2D)          (None, 222, 222, 32)      4640      
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 111, 111, 32)     0         
 2D)                                                             
                                                                 
 conv2d_24 (Conv2D)          (None, 110, 110, 64)      8256      
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 55, 55, 64)       0         
 g2D)                                                            
                                                                 
 dropout_4 (Dropout)         (None, 55, 55, 64)      

In [64]:
# compiling the model
model.compile(optimizer="adam",loss=keras.losses.BinaryCrossentropy,metrics=["accuracy"])

<h1> Model Trainig </h1>

In [85]:
# early stopping and model check point  to save our resources use limitly.
from keras.callbacks import ModelCheckpoint,EarlyStopping
#earlystopping : when model performance stopped improving
es=EarlyStopping(monitor="val_accuracy",  # quantity monitor if not changing with min_delta upto patience no then early stop, dont waste computationl resources.
                 min_delta=0.001,
                 patience=5,
                 verbose=1,
                 restore_best_weights=True # best  waits till point will be in record
                 )
# model saving : making checkpoints by saving tunable weights
# saving best trained model till point
mc=ModelCheckpoint(filepath="~/bestmodel.h5", # used to store huge amount of numerical data in binary format, here store model metadata in.h5 format.
                   monitor="val_accuracy",
                   save_best_only=True,
                   save_weights_only=False,
                   )
tune=[es,mc]

In [86]:
# training model start
# note: model.fit_generator used when dataset is huge, source data is augmenting whie not possible in model.fit case.
# epoch: no of times learning algo will run with traing dataset to train model
histry=model.fit_generator(generator=data_train_augment, #(input,targets) traing data things
                           validation_data=data_val_augment,
                           epochs=300,
                           steps_per_epochs=50, # total data set/ no of batch size
                           verbose=1,
                           callbacks=tune) # list of  callbacks functions applied during traing 


NameError: ignored